In [ ]:
# optional, only for Jupyter
%matplotlib notebook

# General libraries
import numpy as np                # to deal with arrays, vectors, matrices...
import matplotlib.pyplot as plt   # to plot the data
# Tensorflow
import os
HOME = os.getenv('HOME')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # to get rid of the TF compilation warnings
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras.layers import Dense

In [ ]:
# Only because my system-wide config is tuned, you don't need these lines
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = 5,3
mpl.rcParams['font.size'] = 12.0

## Get the data
In this example we load some data manually created. It consists of 2 inputs and 1 output.  

In [ ]:
# Load training and testing data
n_out = 1   # number of outputs

M = np.loadtxt('Spola.train')
IN = M[:,:-n_out]
OUT = M[:,-n_out:].astype(int)

M = np.loadtxt('Spola.test')
IN_test = M[:,:-n_out]
OUT_test = M[:,-n_out:].astype(int)

In [ ]:
print('----> Read data samples <----')
print(f'    Training --> Input: {IN.shape}, Output: {OUT.shape}')
print(f'  Validation --> Input: {IN_test.shape}, Output: {OUT_test.shape}')
inp_shape = IN.shape[1:]

## Visualize the data

In [ ]:
fig = plt.figure()
ax = plt.subplot(111, projection='polar')

# Plot Expected outputs
# this is just a fancy polar plot, the relevant lines are just the scatter
X = IN_test[:,0]
Y = IN_test[:,1]
Z = OUT_test.flatten()
ax.scatter(Y,X,c=Z)  # plot the expected result
ax.set_rmax(36)
ax.set_theta_zero_location("N")
ax.set_title('Expected output',y=1.1)
ax.set_xticklabels(['N', '', 'W', '', 'S', '', 'E', ''])
ax.set_rlabel_position(45)

## Build the model

In [ ]:
model = models.Sequential()
model.add( Dense(3, activation='tanh', input_shape=inp_shape) )
model.add( Dense(5, activation='tanh') )
model.add( Dense(10, activation='tanh') )
model.add( Dense(3, activation='softmax') )  # this dimension should be equal to the number of classes

In [ ]:
model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

### Training

In [ ]:
history = model.fit(IN, OUT, epochs = 100,
                    validation_data = (IN_test,OUT_test),
                    verbose=1)

In [ ]:
err = history.history['loss']
acc = history.history['accuracy']
fig, ax = plt.subplots()
ax.plot(err,label='loss')
ax.plot(acc,label='accuracy')
ax.set_title('Learning curve')
plt.show()

## Check the result

In [ ]:
# Calculate & plot results
OUT_predict = model.predict(IN_test)
OUT_predict = np.argmax(OUT_predict,axis=1)

In [ ]:
fig = plt.figure()
ax0 = plt.subplot(121, projection='polar')   # expected
ax1 = plt.subplot(122, projection='polar')   # predicted

#fig.suptitle("$Neural$ $Network$", fontsize=30)

# Plot Expected outputs
X = IN_test[:,0]
Y = IN_test[:,1]
Z = OUT_test.flatten()
ax0.scatter(Y,X,c=Z)

# Plot Predicted outputs
X = IN_test[:,0]
Y = IN_test[:,1]
Z = OUT_predict.flatten()
ax1.scatter(Y,X,c=Z)  # plot the predicted result

# Making the plot look fancier
ax0.set_rmax(36)
ax0.set_theta_zero_location("N")
ax0.set_title('Expected output',y=1.1)
ax0.set_xticklabels(['N', '', 'W', '', 'S', '', 'E', ''])
ax0.set_rlabel_position(45)

# Settings
ax1.set_rmax(36)
ax1.set_theta_zero_location("N")
ax1.set_title('Predicted output',y=1.1)
ax1.set_xticklabels(['N', '', 'W', '', 'S', '', 'E', ''])
ax1.set_rlabel_position(45)

fig.tight_layout()
plt.show()

In [ ]:
# model.save('spola.h5')